## PARAMS & overview
Ho centralizzato i path e definito gli output principali per preparare i file destinati a Tableau.
Uso questo notebook per: caricare il file cleaned, applicare la rimozione outlier sul prezzo (IQR), creare il file riga-per-riga per la mappa e l'aggregato per dealer_region.

In [21]:
# PARAMS: definisco qui i path canonici (modificare solo se necessario)
import os
import pandas as pd
import numpy as np

# nota: assumo che il notebook sia in project_root/notebook/
ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))  # se il notebook è altrove, aggiusta qui
RAW_DIR = os.path.join(ROOT, "data", "raw")
PROCESSED_DIR = os.path.join(ROOT, "data", "processed")
MAPPINGS_DIR = os.path.join(ROOT, "notebook", "mappings")

# file principali: uso il cleaned canonical fornito da Matteo (o presente in data/processed)
CLEANED_PATH = os.path.join(PROCESSED_DIR, "database_cleaned.csv")
OUT_AGG_PATH = os.path.join(PROCESSED_DIR, "agg_by_dealer_region_for_tableau.csv")
OUT_CITY_PATH = os.path.join(PROCESSED_DIR, "database_for_tableau_city_state.csv")

# controllo rapido — se il file cleaned non esiste prendo nota per ripristino
print("ROOT:", ROOT)
print("CLEANED_PATH exists?", os.path.exists(CLEANED_PATH))
print("PROCESSED_DIR:", PROCESSED_DIR)
print("MAPPINGS_DIR:", MAPPINGS_DIR)

ROOT: /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale
CLEANED_PATH exists? True
PROCESSED_DIR: /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/data/processed
MAPPINGS_DIR: /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/notebook/mappings


In [22]:
# Carico il cleaned file in modo robusto (se esiste la colonna Date la converto)
# Se CLEANED_PATH non è presente, intervenire prima di eseguire le celle successive.
cols0 = pd.read_csv(CLEANED_PATH, nrows=0).columns.tolist()
parse_arg = ['Date'] if 'Date' in cols0 else None

df = pd.read_csv(CLEANED_PATH, parse_dates=parse_arg, low_memory=False)
print("Caricato df shape:", df.shape)
print("Colonne presenti:", df.columns.tolist())
display(df.head(3))

Caricato df shape: (23906, 13)
Colonne presenti: ['Date', 'Customer Name', 'Gender', 'Annual Income', 'Dealer_Name', 'Company', 'Model', 'Engine', 'Transmission', 'Color', 'Price ($)', 'Body Style', 'Dealer_Region']


,Date,Customer Name,Gender,Annual Income,Dealer_Name,Company,Model,Engine,Transmission,Color,Price ($),Body Style,Dealer_Region
0,2022-01-02,Geraldine,Male,13500,Buddy Storbeck's Diesel Service Inc,Ford,Expedition,DoubleÂ Overhead Camshaft,Auto,Black,26000,SUV,Middletown
1,2022-01-02,Gia,Male,1480000,C & M Motors Inc,Dodge,Durango,DoubleÂ Overhead Camshaft,Auto,Black,19000,SUV,Aurora
2,2022-01-02,Gianna,Male,1035000,Capitol KIA,Cadillac,Eldorado,Overhead Camshaft,Manual,Red,31500,Passenger,Greenville


## Pulizia minima e regola outlier
- Trasformo Price e Annual Income in numerici (rimuovo simboli).
- Applico outlier detection **solo su Price** usando la regola IQR (1.5 * IQR).
- Tengo due dataframe:
  - `df` = originale (con outlier) — per audit;
  - `df_no_outliers` = senza outlier sul prezzo — usato per salvare i file per Tableau.

In [23]:
# Pulisco le colonne money e le converto in numerico (mantengo la colonna originale per controllo)
def to_numeric_money(series):
    # rimuovo simboli non numerici, gestisco valori vuoti
    s = series.astype(str).str.replace(r"[^\d\.\-]", "", regex=True)
    s = s.replace("", np.nan)
    return pd.to_numeric(s, errors='coerce')

# riconosco i nomi tipici delle colonne (tolleranza a varianti)
price_cols = [c for c in df.columns if c.lower().strip() in ("price ($)","price","price_$","price($)","price ($)")]
income_cols = [c for c in df.columns if c.lower().strip() in ("annual income","annual_income","income","annualincome")]

if not price_cols:
    raise RuntimeError("Non trovo la colonna Price: controlla i nomi delle colonne nel dataset.")
PRICE_COL = price_cols[0]
INCOME_COL = income_cols[0] if income_cols else None

# creo colonne pulite
# Ho chiamato le colonne _price_clean e _income_clean per non sovrascrivere i dati originali
df["_price_clean"] = to_numeric_money(df[PRICE_COL])
if INCOME_COL:
    df["_income_clean"] = to_numeric_money(df[INCOME_COL])
else:
    df["_income_clean"] = np.nan

print("Pulizia price: valori nulli ->", df["_price_clean"].isna().sum(), 
      "| min/max ->", df["_price_clean"].min(), df["_price_clean"].max())

# Outlier detection SOLO su price (IQR)
q1 = df["_price_clean"].quantile(0.25)
q3 = df["_price_clean"].quantile(0.75)
iqr = q3 - q1
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

# segnalo gli outlier in una colonna booleana; li escludo solo per i file di visualizzazione
df["_price_outlier"] = (df["_price_clean"] < lower) | (df["_price_clean"] > upper)
n_outliers = int(df["_price_outlier"].sum())
print(f"Ho identificato {n_outliers} outlier sul prezzo (IQR).")

# creo df_no_outliers usato per le esportazioni verso Tableau
df_no_outliers = df[~df["_price_outlier"]].copy()
print("Shape df_no_outliers:", df_no_outliers.shape)

Pulizia price: valori nulli -> 0 | min/max -> 1200 85800
Ho identificato 1449 outlier sul prezzo (IQR).
Shape df_no_outliers: (22457, 16)


## Tableau Prep · Sezione finale
Questa sezione produce i dataset aggregati per **Dealer_Region** necessari a costruire in Tableau una mappa del **rapporto tra prezzo medio per auto e reddito medio dell’acquirente**.  
Output previsti:
- `dealer_ratio_for_tableau.csv` → KPI per distretto
- `dealer_ratio_for_tableau_citystate.csv` → KPI per distretto + `city_state_lookup` per geocodifica
- `dealer_ratio_by_year_for_tableau.csv` → variante per analisi temporale anno su anno

FileNotFoundError: File 'database_cleaned_2.csv' non trovato in nessuna delle cartelle candidate: data/processed, data, /mnt/data

### Controlli di qualità essenziali
Si effettua un controllo sintetico su null, valori negativi e numerosità dei distretti.  
L’obiettivo è prevenire problemi in fase di visualizzazione.

In [ ]:
# === QC VELOCE ===
qc = {
    "rows": len(df),
    "cols": len(df.columns),
    "null_rate_price": float(df["Price ($)"].isna().mean()),
    "null_rate_income": float(df["Annual Income"].isna().mean()),
    "neg_values_price": int((df["Price ($)"] < 0).sum()),
    "neg_values_income": int((df["Annual Income"] < 0).sum()),
    "dealer_regions": int(df["Dealer_Region"].nunique()),
}
print("QC summary:", qc)

# Normalizzazione: valori negativi non sono ammessi per prezzo e reddito
df.loc[df["Price ($)"] < 0, "Price ($)"] = np.nan
df.loc[df["Annual Income"] < 0, "Annual Income"] = np.nan

### Aggregazione per Dealer_Region e calcolo del ratio
Si calcolano:
- prezzo medio, mediana prezzo
- reddito medio
- numero di osservazioni per distretto
- **price_to_income_ratio** = avg_price / avg_income
- **ratio_quantile** per una color scale discreta in mappa

Si aggiunge poi un lookup `city_state` per facilitare la geocodifica in Tableau.

In [27]:
# === PATH RESOLUTION (repo-aware) ===

from pathlib import Path
import os

# Nomi file attesi in data/processed
MAIN_NAME = "database_cleaned_2.csv"
CITYSTATE_NAME = "database_for_tableau_city_state.csv"

# Candidati BASE_DIR: se il notebook è in root o dentro /notebook
candidate_roots = [Path("."), Path("..")]

RAW_MAIN = None
CITYSTATE_FILE = None

for root in candidate_roots:
    base_proc = (root / "data" / "processed").resolve()
    base_data = (root / "data").resolve()
    candidates = [
        base_proc / MAIN_NAME,
        base_data / MAIN_NAME,
        Path("/mnt/data") / MAIN_NAME,   # fallback per ambienti remoti
    ]
    for c in candidates:
        if c.exists():
            RAW_MAIN = c
            break
    candidates_city = [
        base_proc / CITYSTATE_NAME,
        base_data / CITYSTATE_NAME,
        Path("/mnt/data") / CITYSTATE_NAME,
    ]
    for c in candidates_city:
        if c.exists():
            CITYSTATE_FILE = c
            break
    if RAW_MAIN is not None:
        break  # abbiamo abbastanza per procedere (CITYSTATE è opzionale)

if RAW_MAIN is None:
    raise FileNotFoundError(
        "database_cleaned_2.csv non trovato. Atteso in 'data/processed' (o 'data'). "
        "Verificare che il notebook sia lanciato dalla root del repo o da /notebook."
    )

print("Path risolti:")
print(" - RAW_MAIN:", RAW_MAIN)
print(" - CITYSTATE_FILE:", CITYSTATE_FILE if CITYSTATE_FILE else "non trovato (lookup opzionale)")

Path risolti:
 - RAW_MAIN: /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/data/processed/database_cleaned_2.csv
 - CITYSTATE_FILE: /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/data/processed/database_for_tableau_city_state.csv


In [29]:
# === LOAD & TYPES ===
# Scopo: caricare i dati puliti e tipizzare i campi critici usati da Tableau.

import pandas as pd
import numpy as np

df = pd.read_csv(RAW_MAIN, low_memory=False)

# Campi richiesti dalla metrica e dalla mappa
required_cols = ["Dealer_Region", "Price ($)", "Annual Income", "Date"]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Colonne mancanti: {missing} nel file {RAW_MAIN.name}")

# Tipi coerenti
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
df["Price ($)"] = pd.to_numeric(df["Price ($)"], errors="coerce")
df["Annual Income"] = pd.to_numeric(df["Annual Income"], errors="coerce")

# (Opzionale) carico il lookup per city/state se disponibile
cs = None
if CITYSTATE_FILE and CITYSTATE_FILE.exists():
    cs = pd.read_csv(CITYSTATE_FILE, low_memory=False)

In [ ]:
# === AGGREGATION · KPI PER MAPPA ===
# Scopo: costruire i KPI per Dealer_Region necessari alla mappa in Tableau.
# Output:
#   - agg_region: KPI per distretto (avg_price, avg_income, ratio, n_obs, quantili)
#   - agg_region_filtered: come sopra + city_state_lookup (se disponibile) e filtro su n_obs

import pandas as pd
import numpy as np
from pathlib import Path

# Safety: la base df deve essere già caricata nella sezione Load (con le colonne richieste)
required = ["Dealer_Region", "Price ($)", "Annual Income"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Mancano colonne per l'aggregazione: {missing}")

# Parametri default nel caso non siano già stati definiti sopra
try:
    MIN_OBS_PER_REGION
except NameError:
    MIN_OBS_PER_REGION = 1
try:
    N_QUANTILES
except NameError:
    N_QUANTILES = 5

# Aggregazione per distretto
agg_region = (
    df.groupby("Dealer_Region", dropna=False)
      .agg(
          avg_price=("Price ($)", "mean"),
          price_median=("Price ($)", "median"),
          avg_income=("Annual Income", "mean"),
          n_obs=("Price ($)", "count"),
      )
      .reset_index()
)

# Ratio e gestione inf/zero
agg_region["price_to_income_ratio"] = agg_region["avg_price"] / agg_region["avg_income"]
agg_region.loc[~np.isfinite(agg_region["price_to_income_ratio"]), "price_to_income_ratio"] = np.nan

# Quantili per color scale discreta
try:
    agg_region["ratio_quantile"] = pd.qcut(
        agg_region["price_to_income_ratio"], q=N_QUANTILES, labels=False, duplicates="drop"
    ) + 1
except ValueError:
    agg_region["ratio_quantile"] = np.nan

# Filtro opzionale su numerosità (riduce rumore in mappa)
agg_region_filtered = agg_region.loc[agg_region["n_obs"] >= MIN_OBS_PER_REGION].copy()

# city_state_lookup (lookup moda per distretto) se disponibile
city_lookup = None
if "cs" in globals() and isinstance(cs, pd.DataFrame) and \
   "Dealer_Region" in cs.columns and "city_state" in cs.columns:
    base_cs = cs
elif "CITYSTATE_FILE" in globals() and CITYSTATE_FILE and Path(CITYSTATE_FILE).exists():
    base_cs = pd.read_csv(CITYSTATE_FILE, low_memory=False)
else:
    base_cs = None

if base_cs is not None and "Dealer_Region" in base_cs.columns and "city_state" in base_cs.columns:
    city_lookup = (
        base_cs.groupby("Dealer_Region", dropna=False)["city_state"]
               .agg(lambda x: x.mode().iat[0] if x.mode().size else np.nan)
               .reset_index()
               .rename(columns={"city_state": "city_state_lookup"})
    )
    agg_region_filtered = agg_region_filtered.merge(city_lookup, on="Dealer_Region", how="left")
else:
    agg_region_filtered["city_state_lookup"] = np.nan

# Controllo rapido
display(agg_region.head(5))
display(agg_region_filtered.head(5))

,Dealer_Region,avg_price,price_median,avg_income,n_obs,price_to_income_ratio,ratio_quantile
0,Aurora,28334.626837,23000.0,834341.026837,3130,0.033960,2
1,Austin,28341.603628,23801.0,809496.730593,4135,0.035011,5
2,Greenville,28180.819054,22500.0,823138.340793,3128,0.034236,4
3,Janesville,27833.350955,23000.0,827446.300183,3821,0.033638,1
4,Middletown,27856.338875,22750.0,818402.594309,3128,0.034037,3


,Dealer_Region,avg_price,price_median,avg_income,n_obs,price_to_income_ratio,ratio_quantile,city_state_lookup
0,Aurora,28334.626837,23000.0,834341.026837,3130,0.033960,2,"Aurora, USA"
1,Austin,28341.603628,23801.0,809496.730593,4135,0.035011,5,"Austin, USA"
2,Greenville,28180.819054,22500.0,823138.340793,3128,0.034236,4,"Greenville, USA"
3,Janesville,27833.350955,23000.0,827446.300183,3821,0.033638,1,"Janesville, USA"
4,Middletown,27856.338875,22750.0,818402.594309,3128,0.034037,3,"Middletown, USA"


In [41]:
# === PATCH Stati per geocodifica ===
# Nota per il team:
# Molte delle città dei dealer district hanno nomi ambigui (negli USA lo stesso nome esiste in più stati).
# Tableau Public Web non consente di risolvere queste ambiguità manualmente, quindi abbiamo scelto
# uno Stato a caso tra le possibilità. Questa scelta è convenzionale e serve solo a permettere
# la geocodifica in Tableau, non riflette la posizione reale dei dealer.

city_to_state = {
    "Aurora": "Illinois",
    "Austin": "Texas",
    "Greenville": "South Carolina",
    "Janesville": "Wisconsin",
    "Middletown": "Connecticut",
    "Pasco": "Washington",
    "Scottsdale": "Arizona"
}

# Aggiungiamo la colonna State basata sulla mappatura manuale
if "City" in agg_region_filtered.columns:
    agg_region_filtered["State"] = agg_region_filtered["City"].map(city_to_state)
else:
    print("Attenzione: la colonna 'City' non esiste in agg_region_filtered. Verificare pipeline.")

In [42]:
# === EXPORT CSV · MAPPA STATICA ===
# Scopo: salvare i dataset finali per Tableau nella cartella ufficiale del repo: data/processed
# Requisiti: 'agg_region' e 'agg_region_filtered' sono stati creati nella cella "AGGREGATION · KPI PER MAPPA".

from pathlib import Path

# 1) Individuazione directory export (root repo → data/processed)
here = Path.cwd().resolve()
candidates = [here, here.parent, here.parent.parent]
repo_root = None
for base in candidates:
    if (base / "data" / "processed").exists():
        repo_root = base
        break
if repo_root is None:
    repo_root = here

OUT_DIR = (repo_root / "data" / "processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 2) Safety check
if "agg_region" not in globals() or "agg_region_filtered" not in globals():
    raise RuntimeError("Aggregazioni mancanti: eseguire prima 'AGGREGATION · KPI PER MAPPA'.")

# 3) Normalizzazione per geocodifica in Tableau 
# - Creiamo due colonne pulite: Country e City (il web authoring geocodifica bene City+Country)
if "city_state_lookup" in agg_region_filtered.columns:
    # uniformiamo il country
    tmp = (
        agg_region_filtered["city_state_lookup"]
        .astype(str)
        .str.strip()
        .str.replace(r",\s*USA$", ", United States", regex=True)
    )
    # City = parte prima della virgola 
    agg_region_filtered["City"] = tmp.str.split(",").str[0].str.strip()
    # Country = parte dopo la virgola, oppure "United States" di default
    c2 = tmp.str.split(",").str[1].fillna("United States").str.strip()
    c2 = c2.replace("", "United States")
    agg_region_filtered["Country"] = c2

    # Drop eventuale colonna “clean” precedente per evitare duplicati/confusione
    if "city_state_lookup_clean" in agg_region_filtered.columns:
        agg_region_filtered.drop(columns=["city_state_lookup_clean"], inplace=True, errors="ignore")

# 4) Salvataggi finali 
out_dist = OUT_DIR / "dealer_ratio_for_tableau.csv"                 # KPI per distretto
out_dist_city = OUT_DIR / "dealer_ratio_for_tableau_citystate.csv"  # KPI + City/Country

agg_region.to_csv(out_dist, index=False)
agg_region_filtered.to_csv(out_dist_city, index=False)

print("Export completato:")
print("→", out_dist.resolve())
print("→", out_dist_city.resolve())

# 4) Salvataggi finali
out_dist = OUT_DIR / "dealer_ratio_for_tableau.csv"                 # KPI per distretto
out_dist_city = OUT_DIR / "dealer_ratio_for_tableau_citystate.csv"  # KPI + city_state_lookup

agg_region.to_csv(out_dist, index=False)
agg_region_filtered.to_csv(out_dist_city, index=False)

print("Export completato:")
print("→", out_dist.resolve())
print("→", out_dist_city.resolve())
print("Export dir:", OUT_DIR.resolve())

Export completato:
→ /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/data/processed/dealer_ratio_for_tableau.csv
→ /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/data/processed/dealer_ratio_for_tableau_citystate.csv
Export completato:
→ /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/data/processed/dealer_ratio_for_tableau.csv
→ /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/data/processed/dealer_ratio_for_tableau_citystate.csv
Export dir: /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/data/processed
